In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

np.random.seed(1)

## Ідея використання
Власне ідея така:
- на початку запуску програми іде генерація юзерів
- згенерована матриця з нулів і одиниць каже подобається(1) чи ні(0).
- після генерації потрібно створити та заповнити стовпчик юзера в програмі на початку запитується які фільми подобаються власне за іменем фільму в стовпчику юзера(нульовому) ставимо одиниці за індексом.
- після матричної факторизації, ми дивимося на останній ствопчик (ствопчик юзера) і шукаємо там числа більші або рівні 0.5, це буде означати, що скоріш за все фільм сподобається користувачу і можна його порекомндувати якщо ми знайшли такий фільм
- тоді беремо його індекс і дивимося його назву і рекомендуємо користувачу (або принаймні додаємо його в список фільмів які ми йому рекомендуємо).

#### Використовуємо то й же датасет 

In [2]:
movie = pd.read_csv('movie.csv')
movie

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015,1.747999e+08,1.385749e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10861,21,tt0060371,0.080598,0,0,The Endless Summer,Michael Hynson|Robert August|Lord 'Tally Ho' B...,NaN,Bruce Brown,NaN,...,"The Endless Summer, by Bruce Brown, is one of ...",95,Documentary,Bruce Brown Films,6/15/1966,11,7.4,1966,0.000000e+00,0.000000e+00
10862,20379,tt0060472,0.065543,0,0,Grand Prix,James Garner|Eva Marie Saint|Yves Montand|Tosh...,NaN,John Frankenheimer,Cinerama sweeps YOU into a drama of speed and ...,...,Grand Prix driver Pete Aron is fired by his te...,176,Action|Adventure|Drama,Cherokee Productions|Joel Productions|Douglas ...,12/21/1966,20,5.7,1966,0.000000e+00,0.000000e+00
10863,39768,tt0060161,0.065141,0,0,Beregis Avtomobilya,Innokentiy Smoktunovskiy|Oleg Efremov|Georgi Z...,NaN,Eldar Ryazanov,NaN,...,An insurance agent who moonlights as a carthie...,94,Mystery|Comedy,Mosfilm,1/1/1966,11,6.5,1966,0.000000e+00,0.000000e+00
10864,21449,tt0061177,0.064317,0,0,"What's Up, Tiger Lily?",Tatsuya Mihashi|Akiko Wakabayashi|Mie Hama|Joh...,NaN,Woody Allen,WOODY ALLEN STRIKES BACK!,...,"In comic Woody Allen's film debut, he took the...",80,Action|Comedy,Benedict Pictures Corp.,11/2/1966,22,5.4,1966,0.000000e+00,0.000000e+00


#### Моделюємо оцінка користувачів друзів

In [24]:
from scipy.linalg import cholesky

# n_films = 50
# n_users = 5
# Correlation matrix
# corr_mat= np.array([[1.0, 0.6, 0.4, 0.7, 0.8],
#                     [0.6, 1.0, 0.65, 0.4, 0.7],
#                     [0.4, 0.65, 1.0, 0.3, 0.85],
#                     [0.7, 0.4, 0.3, 1.0, 0.6],
#                     [0.8, 0.7, 0.85, 0.6, 1]])

# Compute the (upper) Cholesky decomposition matrix
# upper_chol = cholesky(corr_mat)

# Generate 3 series of normally distributed (Gaussian) numbers
# rnd = np.random.normal(0.0, 0, size=(n_films, n_users))

# # Finally, compute the inner product of upper_chol and rnd
# ans = rnd @ upper_chol
# rand_correlated_sample = abs(ans)
# rand_correlated_sample[rand_correlated_sample > 0] = 1
# rand_correlated_sample[rand_correlated_sample <= 0] = 0
# rand_correlated_sample = np.around(rand_correlated_sample)
# rand_correlated_sample

arr = np.random.uniform(0, 1, size=(len(movie), 5))
for i in range(len(arr)):
    for j in range(5):
        if arr[i][j] >= 0.5:
            arr[i][j] = 1
        else:
            arr[i][j] = 0

user = np.zeros((len(movie), 1))
user[23][0] = 1
user[12][0] = 1
user[101][0] = 1
user[222][0] = 1
user[41][0] = 1
user[501][0] = 1
user[600][0] = 1

users = np.append(arr, user, axis=1)

In [28]:
n_users = 6
V = users
print('Сума згенерованих оцінок: ', V.sum())
V

Сума згенерованих оцінок:  27211.0


array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 1., 0.],
       [1., 1., 0., 1., 1., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0.],
       [0., 1., 0., 1., 1., 0.]])

#### Ініціалізуємо невід'ємну матричну факторизацію (Non-Negative Matrix Factorization(NMF))
**Алгоритм невд'ємної матричної факторизації** - це алгоритм, що полягає в розкладенні матриці $V$ на добуток матриць $W$ та $H$, де кожна з матриць містить невд'ємні елементи. Це розкладення володіє багатьма чудовими властивостями, але головна властивість така, що після відновлення матриці V, ми не отримаємо її в початковому вигляді, а здеякими збуреннями. Ці збурення нам і вкажуть де можуть знаходитись предмети рекомендації. Це називають внутрішньою властивістю кластеризації, тобто це означає, що алгоритм автоматично групує стовпці матриці $V$<br>


In [29]:
nmf = NMF(n_components=2, init='random', random_state=0, max_iter=200, shuffle=True)
nmf.fit(V)

NMF(alpha=0.0, beta_loss='frobenius', init='random', l1_ratio=0.0, max_iter=200,
    n_components=2, random_state=0, shuffle=True, solver='cd', tol=0.0001,
    verbose=0)

#### Тепер проводимо знаходимо матриці H та W і для того щоб отримати апроксимацію - перемножуємо їх
https://en.wikipedia.org/wiki/Non-negative_matrix_factorization#:~:text=Non%2Dnegative%20matrix%20factorization%20

In [31]:
H = pd.DataFrame(np.round(nmf.components_, n_users))

W = pd.DataFrame(np.round(nmf.transform(V), n_users))

reconstructed = pd.DataFrame(np.round(np.dot(W,H), 2), columns=['user1', 'user2', 'user3', 'user4', 'user5', 'user6'])

In [33]:
print('Тепер така сума нашого оновленого стовпчика:')
print(reconstructed.sum())

Тепер така сума нашого оновленого стовпчика:
user1    5495.03
user2    5454.77
user3    5427.17
user4    5506.35
user5    5376.27
user6       0.00
dtype: float64



---

#### Тепер замість чисел стоять числа від 0 до 1
Можна було запринтити без приведення до інтервалу 0-1, але так наглядніше.<br>
Тепер навіть на місці елементів де були нулі(людина ще не дивилась цей фільм).<br>
Алгоритм видає приблизну оцінку чи сподобається фільм людині, чи ні.<br>

In [35]:
reconstructed

,user1,user2,user3,user4,user5,user6
0,0.00,0.25,0.25,0.24,0.24,0.0
1,0.00,0.51,0.51,0.49,0.50,0.0
2,1.01,0.74,0.74,0.77,0.74,0.0
3,0.00,0.26,0.26,0.25,0.25,0.0
4,1.00,0.24,0.24,0.27,0.24,0.0
...,...,...,...,...,...,...
10861,0.02,0.76,0.76,0.73,0.74,0.0
10862,0.00,0.26,0.26,0.25,0.25,0.0
10863,0.00,0.26,0.26,0.25,0.25,0.0
10864,0.02,0.76,0.75,0.73,0.74,0.0


нажаль результатів не отримали, але чим більше користувач буде додавати фільмів
ти краще буде працювати матрична факторизація(так як алгоритм буде бачити які
юзери корелюють між собою)